# <center>Firestoreにデータを置く</center>

In [1]:
# 使用するライブラリの読み込み
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from google.cloud.firestore import GeoPoint
import pandas as pd

/Users/msz/.pyenv/versions/3.8.7/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
# pandasのDataFrameでcsvからデータを読み込む
df = pd.read_csv("データを保存しているCSVファイル",encoding="utf-8")
# 東京都に絞り込む
df = df[df["shop_place"].str.contains("東京都")]

In [6]:
# 各種リストの作成
id_list = df["level_0"].to_list()
title_list = df["title"].to_list()
geo_n = df["ido"].to_list()
geo_t = df["kdo"].to_list()
genre = df["genre"].to_list()
youtube_link = df["link"].to_list()
youtube_id = [yl.split("v=")[1] for yl in youtube_link]
shop_name = df["shop_name"].to_list()
shop_link = df["shop_link"].to_list()
shop_place = df["shop_place"].to_list()
img_path = df["image_path"].to_list()

In [7]:
genre_dict = {"家系":"iekei",
              "二郎系":"jiro",
              "激辛":"kara",
              "つけ麺":"tsuke",
              "まぜそば":"maze"}

In [28]:
# firestoreに投げるためにデータの形を整形する
genre_tmp = genre[0]
all_dict_list = []
dict_list = []
for i,id_ in enumerate(id_list):
    if genre[i] != genre_tmp:
        print(genre[i])
        genre_tmp = genre[i]
        all_dict_list.append(dict_list)
        dict_list = []
    
    dict_list.append({u'id':id_list[i],
                    u'title':title_list[i],
                    u'lat':geo_n[i],
                    u'lng':geo_t[i],
                    u'youtube_link':youtube_link[i],
                    u'youtube_id':youtube_id[i],
                    u'shop_name':shop_name[i],
                    u'shop_place':shop_place[i],
                    u'shop_link':shop_link[i]})
all_dict_list.append(dict_list)

激辛
つけ麺
まぜそば
家系


In [29]:
genre_list = ["jiro","kara","tsuke","maze","iekei"]

In [26]:
"""
firestoreデータにアクセス
"""

# 秘密鍵によるfirebaseに認証を行う
cred = credentials.Certificate("発行された秘密鍵のjsonファイル名")
firebase_admin.initialize_app(cred)

# firestoreにアクセスする
db = firestore.client()

"""
firestoreにデータを書き込む
"""
for i,id_ in enumerate(genre_list):
    # firestoreのtemplesのid_(1,2,3,4,...)にアクセス
    ref = db.collection("shop_data").document(id_)
    # 各種データを書き込む
    ref.set({u"data":all_dict_list[i]})